# Optimization - CEM

<img src="https://raw.githubusercontent.com/jeremiedecock/polytechnique-inf581-2021/master/logo.jpg" style="float: left; width: 15%" />

[INF581-2021](https://moodle.polytechnique.fr/course/view.php?id=9352) Lab session #8

2019-2021 Jérémie Decock

[![Open in Google Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jeremiedecock/polytechnique-inf581-2021/blob/master/lab8_optim_cem_answers.ipynb)

[![My Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jeremiedecock/polytechnique-inf581-2021/master?filepath=lab8_optim_cem_answers.ipynb)

[![NbViewer](https://raw.githubusercontent.com/jupyter/design/master/logos/Badges/nbviewer_badge.svg)](https://nbviewer.jupyter.org/github/jeremiedecock/polytechnique-inf581-2021/blob/master/lab8_optim_cem_answers.ipynb)

[![Local](https://img.shields.io/badge/Local-Save%20As...-blue)](https://github.com/jeremiedecock/polytechnique-inf581-2021/raw/master/lab8_optim_cem_answers.ipynb)

**Notice**: this notebook requires the following libraries: OpenAI *Gym*, NumPy, Pandas, Seaborn and imageio.

You can install them with the following command (the next cells do this for you if you use the Google Colab environment):

``
pip install gym numpy pandas seaborn imageio
``

In [ ]:
%matplotlib inline

import subprocess

try:
    from inf581 import *
except ModuleNotFoundError:
    process = subprocess.Popen("pip install inf581".split(), stdout=subprocess.PIPE)

    for line in process.stdout:
        print(line.decode().strip())

    from inf581 import *

import matplotlib.pyplot as plt

import gym
import numpy as np
import seaborn as sns

from IPython.display import Image   # To display GIF images in the notebook

In [ ]:
sns.set_context("talk")

$
\newcommand{\vs}[1]{\mathbf{#1}} % vector symbol (\boldsymbol, \textbf or \vec)
\newcommand{\ms}[1]{\mathbf{#1}} % matrix symbol (\boldsymbol, \textbf)
\def\U{V}
\def\action{\vs{a}}       % action
\def\A{\mathcal{A}}        % TODO
\def\actionset{\mathcal{A}} %%%
\def\discount{\gamma}  % discount factor
\def\state{\vs{s}}         % state
\def\S{\mathcal{S}}         % TODO
\def\stateset{\mathcal{S}}  %%%
%
\def\E{\mathbb{E}}
%\newcommand{transition}{T(s,a,s')}
%\newcommand{transitionfunc}{\mathcal{T}^a_{ss'}}
\newcommand{transitionfunc}{P}
\newcommand{transitionfuncinst}{P(\nextstate|\state,\action)}
\newcommand{transitionfuncpi}{\mathcal{T}^{\pi_i(s)}_{ss'}}
\newcommand{rewardfunc}{r}
\newcommand{rewardfuncinst}{r(\state,\action,\nextstate)}
\newcommand{rewardfuncpi}{r(s,\pi_i(s),s')}
\newcommand{statespace}{\mathcal{S}}
\newcommand{statespaceterm}{\mathcal{S}^F}
\newcommand{statespacefull}{\mathcal{S^+}}
\newcommand{actionspace}{\mathcal{A}}
\newcommand{reward}{R}
\newcommand{statet}{S}
\newcommand{actiont}{A}
\newcommand{newstatet}{S'}
\newcommand{nextstate}{\state'}
\newcommand{newactiont}{A'}
\newcommand{stepsize}{\alpha}
\newcommand{discount}{\gamma}
\newcommand{qtable}{Q}
\newcommand{finalstate}{\state_F}
%
\newcommand{\vs}[1]{\boldsymbol{#1}} % vector symbol (\boldsymbol, \textbf or \vec)
\newcommand{\ms}[1]{\boldsymbol{#1}} % matrix symbol (\boldsymbol, \textbf)
\def\vit{Value Iteration}
\def\pit{Policy Iteration}
\def\discount{\gamma}  % discount factor
\def\state{\vs{s}}         % state
\def\S{\mathcal{S}}         % TODO
\def\stateset{\mathcal{S}}  %%%
\def\cstateset{\mathcal{X}} %%%
\def\x{\vs{x}}                    % TODO cstate
\def\cstate{\vs{x}}               %%%
\def\policy{\pi}
\def\piparam{\vs{\theta}}         % TODO pparam
\def\action{\vs{a}}       % action
\def\A{\mathcal{A}}        % TODO
\def\actionset{\mathcal{A}} %%%
\def\caction{\vs{u}}       % action
\def\cactionset{\mathcal{U}} %%%
\def\decision{\vs{d}}       % decision
\def\randvar{\vs{\omega}}       %%%
\def\randset{\Omega}       %%%
\def\transition{T}       %%%
\def\immediatereward{r}    %%%
\def\strategichorizon{s}    %%% % TODO
\def\tacticalhorizon{k}    %%%  % TODO
\def\operationalhorizon{h}    %%%
\def\constalpha{a}    %%%
\def\U{V}              % utility function
\def\valuefunc{V}
\def\X{\mathcal{X}}
\def\meu{Maximum Expected Utility}
\def\finaltime{T}
\def\timeindex{t}
\def\iterationindex{i}
\def\decisionfunc{d}       % action
\def\mdp{\text{MDP}}
$

## Introduction

In the previous lab we studied a method that allowed us to apply reinforcement learning in continuous state spaces and/or continuous action spaces.
We used REINFORCE, a *Policy gradient* method that directly optimize the parametric policy $\pi_{\theta}$.
The parameter $\theta$ was iteratively updated toward a local maximum of the total expected reward $J(\theta)$ using a gradient ascent method:
$$\theta \leftarrow \theta + \alpha \nabla_{\theta}J(\theta)$$
A convenient analytical formulation of $\nabla_{\theta}J(\theta)$ was obtained thanks to the *Policy Gradient theorem*:

$$\nabla_\theta J(\theta) = \nabla_\theta V^{\pi_\theta}(s) = \E_{\pi_\theta} \left[\nabla_\theta \log \pi_\theta (s,a) Q^{\pi_\theta}(s,a) \right].$$
However, gradient ascent methods may have a slow convergence and will only found a local optimum.
Moreover, this approach requires an analytical formulation of $\nabla_\theta \log \pi_\theta (s,a)$ which is not always known (when something else than a neural networks is used for the agent's policy).

Direct Policy Search methods using gradient free optimization procedures like CEM are interesting alternatives to Policy Gradient algorithms.
They can be successfully applied as long as the $\pi_\theta$ policy has no more than few hundreds of parameters.
Moreover, these method can solve complex problems that cannot be modeled as Markov Decision Processes.

As for previous Reinforcement Learning labs, we will use standard problems provided by OpenAI Gym suite.
Especially, we will try to solve the MountainCarContinuous-v0 problem (\url{https://github.com/openai/gym/wiki/MountainCarContinuous-v0}) which offers both continuous space and action states.

An underpowered car must climb a one-dimensional hill to reach a target.
The target is on top of a hill on the right-hand side of the car. If the car reaches it or goes beyond, the episode terminates.
On the left-hand side, there is another hill. Climbing this hill can be used to gain potential energy and accelerate towards the target. On top of this second hill, the car cannot go further than a position equal to -1, as if there was a wall. Hitting this limit does not generate a penalty. The problem is considered solved if a reward of 90 is obtained. 

## Exercise 1: Hands on the mountain car environment

**Task 1:** read  https://gym.openai.com/envs/CartPole-v0/ and https://github.com/openai/gym/wiki/CartPole-v0 to discover the CartPole environment.

**Notice:** A reminder of Gym main concepts is available at https://gym.openai.com/docs/.

Print some information about the environment:

In [ ]:
env = gym.make('CartPole-v0')
print("State space dimension is:", env.observation_space.shape[0])
print("State upper bounds:", env.observation_space.high)
print("State lower bounds:", env.observation_space.low)
print("Actions are: {" + ", ".join([str(a) for a in range(env.action_space.n)]) + "}")
env.close()

**Task 2:** Run the following cells and check different basic 

policies (for instance constant actions or randomly drawn actions) to discover the CartPole environment.
Although this environment has easy dynamics that can be computed analytically, we will solve this problem with Policy Gradient based Reinforcement learning.

### Test the CartPole environment with a constant policy

In [ ]:
env = gym.make('CartPole-v0')
RenderWrapper.register(env, force_gif=True)

observation = env.reset()
done = False

for t in range(50):
    env.render_wrapper.render()

    if not done:
        print(observation)
    else:
        print("x", end="")

    ### BEGIN SOLUTION ###

    action = 0
    observation, reward, done, info = env.step(action)

    ### END SOLUTION ###


print()
env.close()

env.render_wrapper.make_gif("ex1left")

In [ ]:
env = gym.make('CartPole-v0')
RenderWrapper.register(env, force_gif=True)

observation = env.reset()
done = False

for t in range(50):
    env.render_wrapper.render()

    if not done:
        print(observation)
    else:
        print("x", end="")

    ### BEGIN SOLUTION ###

    action = 1
    observation, reward, done, info = env.step(action)

    ### END SOLUTION ###

print()
env.close()

env.render_wrapper.make_gif("ex1right")

### Test the CartPole environment with a random policy

In [ ]:
env = gym.make('CartPole-v0')
RenderWrapper.register(env, force_gif=True)

for episode_index in range(5):
    observation = env.reset()
    done = False

    for t in range(70):
        env.render_wrapper.render()

        if not done:
            print(observation)
        else:
            print("x", end="")
        
        ### BEGIN SOLUTION ###

        action = env.action_space.sample()
        observation, reward, done, info = env.step(action)

        ### END SOLUTION ###

    print()
    env.close()

env.render_wrapper.make_gif("ex1random")

## Exercise 2: Implement a sigmoid policy

As the number of actions is $2$ (push the cart to the left or push it to the right), one can see the problem of controlling the Cart Pole as a binary classification problem.
Binary classification can be easily solved thanks to logistic regression which transforms the classification problem into a regression problem using the sigmoid function.

**Task 1**: Implement the `sigmoid` function defined by:

$$\sigma(x) = \frac{1}{1+e^{-x}}.$$

In [ ]:
def sigmoid(x):
    ### BEGIN SOLUTION ###
    return 1.0 / (1.0 + np.exp(-x))
    ### END SOLUTION ###

**Task 2**: Complete the `logistic_regression` function that implements the logistic regression. This function returns the probability to draw action 1 ("push right") w.r.t the parameter vector $\theta$ and the input vector $s$ (the 4-dimension state vector).

In [ ]:
plot_logistic_regression_fig()

In [ ]:
def logistic_regression(s, theta):
    ### BEGIN SOLUTION ###
    prob_push_right = sigmoid(np.dot(s, np.transpose(theta)))
    ### END SOLUTION ###

    return prob_push_right

**Task 3**: Complete the `draw_action` function that draw an action according to current policy i.e. that select the *RIGHT* action with probability $\sigma(\theta^\top s)$, where $\theta$ is the parameter vector and $s$ is the 4-dimension state vector.

In [ ]:
def draw_action(s, theta):
    prob_push_right = logistic_regression(s, theta)

    ### BEGIN SOLUTION ###

    r = np.random.rand()
    if r < prob_push_right:
        return 1
    else:
        return 0

    ### END SOLUTION ###

## Exercise 3: Compute $\nabla_{\theta} \log \pi_\theta (s,a)$

Verify that, for a sigmoid policy:
- $\nabla_\theta \log \pi_\theta (s,\text{RIGHT}) = \pi_\theta (s, \text{LEFT}) \times s$
- $\nabla_\theta \log \pi_\theta (s,\text{LEFT}) = - \pi_\theta (s, \text{RIGHT}) \times s$ 

### Answer

First, let's define $\pi(s, RIGHT)$ and $\pi(s, LEFT)$:
- $\pi(s, RIGHT) = \sigma(s^\top \theta)$
- $\pi(s, LEFT) = 1 - \sigma(s^\top \theta)$

Then, let's compute the derivative of the sigmoid function:
$$
\sigma'(x) = \sigma(x) (1 - \sigma(x))
$$

1. Compute $\nabla_\theta \log \pi_\theta (s,\text{RIGHT})$:

\begin{align}
\nabla_\theta \log \pi_\theta (s,\text{RIGHT})
&= \nabla_\theta \log \sigma(s^\top \theta) \\
&= \frac{1}{\sigma(s^\top \theta)} \nabla_\theta \sigma(s^\top \theta) \\
&= \frac{1}{\sigma(s^\top \theta)} \sigma(s^\top \theta) (1 - \sigma(s^\top \theta)) ~ s \\
&= (1 - \sigma(s^\top \theta)) ~ s \\
&= \pi_\theta(s, \text{LEFT}) \times s \\
\end{align}

2. Compute $\nabla_\theta \log \pi_\theta (s,\text{LEFT})$:

\begin{align}
\nabla_\theta \log \pi_\theta (s,\text{LEFT})
&= \nabla_\theta \log (1 - \sigma(s^\top \theta)) \\
&= \frac{1}{1 - \sigma(s^\top \theta)} \nabla_\theta (1 - \sigma(s^\top \theta)) \\
&= \frac{-1}{1 - \sigma(s^\top \theta)} \sigma(s^\top \theta) (1 - \sigma(s^\top \theta)) ~ s \\
&= -\sigma(s^\top \theta) ~ s \\
&= -\pi_\theta(s, \text{RIGHT}) \times s \\
\end{align}

## Exercise 4: Implement REINFORCE

Fill the following cells to implement the REINFORCE algorithm (defined in the introduction of this notebook).

In [ ]:
ENV_NAME = "CartPole-v0"

# Since the goal is to attain an average return of 195, horizon should be larger than 195 steps (say 300 for instance)
EPISODE_DURATION = 300

ALPHA_INIT = 0.1
SCORE = 195.0
NUM_EPISODES = 100
LEFT = 0
RIGHT = 1

VERBOSE = True

**Task 1**: Implement the `play_one_episode` function that plays an episode with the given policy $\pi_\theta$ (for fixed horizon $H$) and returns its rollouts.

In [ ]:
# Generate an episode
def play_one_episode(env, theta, max_episode_length=EPISODE_DURATION, render=False):
    s_t = env.reset()

    episode_states = []
    episode_actions = []
    episode_rewards = []
    episode_states.append(s_t)

    for t in range(max_episode_length):

        if render:
            env.render_wrapper.render()

        ### BEGIN SOLUTION ###

        a_t = draw_action(s_t, theta)
        s_t, r_t, done, info = env.step(a_t)

        ### END SOLUTION ###

        episode_states.append(s_t)
        episode_actions.append(a_t)
        episode_rewards.append(r_t)

        if done:
            break

    return episode_states, episode_actions, episode_rewards

**Task 2**: Implement the `score_on_multiple_episodes` function that test the given policy $\pi_\theta$ on `num_episodes` episodes (for fixed horizon $H$) and returns:
- `success`: `True` if the agent got an average reward greater or equals to 195 over 100 consecutive trials, `False` otherwise
- `num_success`: the number of episodes where the agent got an average reward greater or equals to 195
- `average_return`: the average reward on the `num_episodes` episodes

In [ ]:
def score_on_multiple_episodes(env, theta, score=SCORE, num_episodes=NUM_EPISODES, max_episode_length=EPISODE_DURATION, render=False):
    
    ### BEGIN SOLUTION ###
    
    num_success = 0
    average_return = 0
    num_consecutive_success = [0]

    for episode_index in range(num_episodes):
        _, _, episode_rewards = play_one_episode(env, theta, max_episode_length, render)

        total_rewards = sum(episode_rewards)

        if total_rewards >= score:
            num_success += 1
            num_consecutive_success[-1] += 1
        else:
            num_consecutive_success.append(0)

        average_return += (1.0 / num_episodes) * total_rewards

        if render:
            print("Test Episode {0}: Total Reward = {1} - Success = {2}".format(episode_index,total_rewards,total_rewards>score))

    if max(num_consecutive_success) >= 100:    # MAY BE ADAPTED TO SPEED UP THE LERNING PROCEDURE
        success = True
    else:
        success = False
        
    ### END SOLUTION ###

    return success, num_success, average_return

**Task 3**: Implement the `compute_policy_gradient` function that returns Policy Gradient for a given episode: policy gradient = $\sum_{t=1}^H \nabla_\theta \log \pi_\theta(s_t,a,_t) R_t$.

In [ ]:
# Returns Policy Gradient for a given episode
def compute_policy_gradient(episode_states, episode_actions, episode_rewards, theta):

    ### BEGIN SOLUTION ###

    H = len(episode_rewards)
    PG = 0

    for t in range(H):

        èè = logistic_regression(episode_states[t], theta)
        a_t = episode_actions[t]
        R_t = sum(episode_rewards[t::])
        if a_t == LEFT:
            g_theta_log_pi = - prob_push_right * episode_states[t] * R_t
        else:
            prob_push_left = (1 - prob_push_right)
            g_theta_log_pi = prob_push_left * episode_states[t] * R_t

        PG += g_theta_log_pi

    ### END SOLUTION ###

    return PG

**Task 4**: Implement the `train` function that updates $\theta$ parameters with gradient ascent until the agent got an average reward greater or equals to 195 over 100 consecutive trials.

In [ ]:
# Train the agent got an average reward greater or equals to 195 over 100 consecutive trials
def train(env, theta_init, max_episode_length = EPISODE_DURATION, alpha_init = ALPHA_INIT):

    theta = theta_init
    episode_index = 0
    average_returns = []

    success, _, R = score_on_multiple_episodes(env, theta)
    average_returns.append(R)

    # Train until success
    while (not success):

        ### BEGIN SOLUTION ###

        # Rollout
        episode_states, episode_actions, episode_rewards = play_one_episode(env, theta, max_episode_length)

        # Schedule step size
        #alpha = alpha_init
        alpha = alpha_init / (1 + episode_index)

        # Compute gradient
        PG = compute_policy_gradient(episode_states, episode_actions, episode_rewards, theta)

        # Do gradient ascent
        theta += alpha * PG

        # Test new policy
        success, _, R = score_on_multiple_episodes(env, theta, render=False)

        # Monitoring
        average_returns.append(R)

        episode_index += 1

        if VERBOSE:
            print("Episode {0}, average return: {1}".format(episode_index, R))

        ### END SOLUTION ###

    return theta, episode_index, average_returns

### Train the agent

In [ ]:
#np.random.seed(1234)
env = gym.make(ENV_NAME)
RenderWrapper.register(env, force_gif=True)
#env.seed(1234)

In [ ]:
dim = env.observation_space.shape[0]

# Init parameters to random
theta_init = np.random.randn(1, dim)

# Train the agent
theta, i, average_returns = train(env, theta_init)

print("Solved after {} iterations".format(i))

### Test final policy

In [ ]:
score_on_multiple_episodes(env, theta, num_episodes=10, render=True)
env.render_wrapper.make_gif("ex4")

### Display the evolution of the average reward w.r.t. PG iterations

In [ ]:
# Show training curve
plt.plot(range(len(average_returns)),average_returns)
plt.title("Average reward on 100 episodes")
plt.xlabel("Training Steps")
plt.ylabel("Reward")

plt.show()

env.close()

## Bonus: solve more challenging environments using Stable-Baselines

Knowing main Reinforcement Learning concepts is important to solve RL problems, but being able to quickly solve problems reusing robust and proven implementations of RL algorithms is important too.

Stable-Baselines is a Python library that provide state of the art ready to use RL algorithms.

The following notebook provides an example of how to use it with Google Colab: https://colab.research.google.com/github/jeremiedecock/polytechnique-inf581-2021/blob/master/lab7_rl3_reinforce_baselines.ipynb